# Fetching data from BigQuery



The first set of code is from a notebook at https://colab.research.google.com/notebooks/bigquery.ipynb#scrollTo=SeTJb51SKs_W 

This code block gives you a link to log in and authenticate with your Google account. However, it's not clear if that account has to be the one hosting the BigQuery data.

In [1]:
#code from https://colab.research.google.com/notebooks/bigquery.ipynb#scrollTo=SeTJb51SKs_W
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Add interactive tables

The next code means you can create interactive tables from imported data, including filters and sorting.

In [2]:
#creates interactive tables in the notebook with dropdown lists etc.
%load_ext google.colab.data_table

Once that's added you can then run a SQL query and the results are shown in that interactive format.

The part after `--project` needs to match the ID of your BigQuery project - in this case `datacamp-287416`

Likewise the `FROM` part needs to name a table in that BigQuery.

In [ ]:
# Display query output immediately

%%bigquery --project  datacamp-287416 
SELECT 
  COUNT(*) as total_rows
FROM `datacamp-287416.tweetsatmps_month1.alltweets`

,total_rows
0,987578


## Getting data from a different table

That table is the first scrape - but there are others. Here's a query on one. 

In [ ]:
# Display query output immediately

%%bigquery --project  datacamp-287416 
SELECT 
  COUNT(*) as total_rows
FROM `datacamp-287416.tweetsatmps_month1.tweetsatmps_mar14_mar23`

,total_rows
0,508199


And the next one.

In [ ]:
# Display query output immediately

%%bigquery --project  datacamp-287416 
SELECT 
  COUNT(*) as total_rows
FROM `datacamp-287416.tweetsatmps_month1.tweetsatmps_mar23_mar30`

,total_rows
0,617261


## Storing results of a query in a dataframe

You can store those results, rather than showing them, by adding a name for the variable after the project ID - in this case we've used `df`

In [ ]:
# Save output in a variable `df`

%%bigquery --project datacamp-287416 df
SELECT 
  tweet 
FROM `datacamp-287416.tweetsatmps_month1.ortweetsTha_East`
WHERE tweet LIKE "%OliverDowden%"

In [ ]:
#show the first few results
df.head()

,tweet
0,@LouiseM60618083 @Tracey_utv @BorisJohnson @co...
1,@ckcgunner @Tracey_utv @BorisJohnson @columeas...
2,@RommelOuterSpac @Tracey_utv @BorisJohnson @co...
3,@LouiseM60618083 @Tracey_utv @BorisJohnson @co...
4,@OliverDowden @Conservatives @LukeHall @Justin...


In [ ]:
#export to csv
df.to_csv("dowden.csv")

## Create a temporary URL

See https://cloud.google.com/storage/docs/access-control/making-data-public#storage-make-object-public-python

see also https://cloud.google.com/storage/docs/access-control/signing-urls-with-helpers#code-samples

## Create a loop for multiple searches

Can we set up a loop to create a dataframe (and then CSV) for each account?

This approach from the [documentation for the bigquery library](https://googleapis.dev/python/bigquery/latest/usage/client.html) seems to work.

In [ ]:
#import the bigquery library
from google.cloud import bigquery

In [ ]:
#this line below from https://googleapis.dev/python/bigquery/latest/usage/client.html
#found project ID at https://console.cloud.google.com/storage/settings;tab=project_access?authuser=1&project=datacamp-287416
client = bigquery.Client(project='datacamp-287416')

# Perform a query.
QUERY = (
    'SELECT tweet FROM `tweetsatmps_month1.alltweets`'
    'WHERE tweet LIKE "%DavidLammy%"'
    'LIMIT 10')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row.tweet)

@Femi_Sorry Did you call out @DavidLammy for calling members of the ERG as being Nazis?   https://t.co/oAdDHeCER6
@NickCooperative @zarahsultana @DavidLammy Munira Mirza's revolution was 180 degrees.
@zarahsultana @DavidLammy Principal ghostwriter for the Sewell Report?
@zarahsultana @DavidLammy @threadreaderapp unroll
@zarahsultana @DavidLammy #JohnsonOut #ToryCorruption
@sbremner98 @zarahsultana @DavidLammy No she wasn't
@txiguiro @zarahsultana @DavidLammy Parody account spotted.
@LongRagnar @zarahsultana @DavidLammy OK. And only just noticed the date of the tweet, so sorry for digging up old tweets.
@zarahsultana @DavidLammy @threadreaderapp unroll
@NickCooperative @zarahsultana @DavidLammy Munira Mirza's revolution was 180 degrees.


For the other two datasets, the header row wasn't correctly parsed, so we need to change 'tweets' to 'string_field_10'.

In [ ]:
#this line below from https://googleapis.dev/python/bigquery/latest/usage/client.html
#found project ID at https://console.cloud.google.com/storage/settings;tab=project_access?authuser=1&project=datacamp-287416
client = bigquery.Client(project='datacamp-287416')

# Perform a query.
QUERY = (
    'SELECT string_field_10 FROM `tweetsatmps_month1.tweetsatmps_mar14_mar23`'
    'WHERE string_field_10 LIKE "%DavidLammy%"'
    'LIMIT 10')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row.string_field_10)

For @AtlanticCouncil I was proud to host @BernieSanders, @DavidLammy and @delarabur to discuss what a structural response to Putin’s war can be: in terms of decarbonisation and anti-corruption. You can watch the recording below:
The UK must stop #TigrayGenocide. 1/2 million people perished due 🇪🇹&amp; 🇪🇷 govt's blockage, mass rape, killings, disappearance. UK needs to act NOW.  @BorisJohnson @Ed_Miliband @jeremycorbyn @AngelaRayner @vincecable @HackneyAbbott   @ChukaUmunna @CarolineLucas  @DavidLammy
@MartinBarrow I cant get my head around how this was allowed to happen. Teachers abandoned duty of care. MPS showed zero empathy even though they are apparently legally allowed to do the search. No family member called prior to the search?  What about her mental health? @DrRosena @DavidLammy
Please don’t forget that most men are 100% on your side. The leadership in the @UKLabour &amp; the likes of @CrispinBlunt have misjudged the mood of ordinary people spectacularly. @Keir_Starmer  @David

In [ ]:
#create a list of accounts
accountlist = ['columeastwood', 'OliverDowden']

for i in accountlist:
  # Save output in a variable 
  likestring = "%"+i+"%"
  # Perform a query.
  QUERY = (
      'SELECT tweet FROM `tweetsatmps_month1.alltweets`'
      'WHERE tweet LIKE "'+likestring+'"'
      'LIMIT 10')
  query_job = client.query(QUERY)  # API request
  rows = query_job.result()  # Waits for query to finish

  for row in rows:
      print(row.tweet)

@sarahchivers10 @ethaninflux @columeastwood English hate English, go to  football match with 2 local clubs.  Here's a good read, be sure to get to the bit where the Irish Police said not on our doorstep &amp; afterwards Gardai bet the shit out of them !!!   https://t.co/SW7r4nC1lm  https://t.co/N6zjLZZquq
@sarahchivers10 @ethaninflux @columeastwood Was more blood on the ferry than a slaughter house, shipped straight out, weren't aloud into hospitals here 🤣🤣🤣  https://t.co/krgAKYuR1E
@LoveRascal2 @columeastwood @SDLPlive We had that. No evidence.
@MisterTotti @SDLPlive @columeastwood @NicholaMallon @JustinMcNu1ty @ClaireHanna @doloreskelly @ConorHouston_ @PatCatney @McCrossanMLA @patsymcglone @CaraHunterMLA Nonsense. Do your homework
@JohnnyMercerUK @columeastwood The paratroopers weren't just responsible for the deaths at ballymurphy and Derry.  Their actions are what give the IRA the strength to take the war to the brits. Culminating in a British surrender.   https://t.co/LV6qRDiuDQ
@

Can we get more than the tweets?

In [ ]:
#create a list of accounts
accountlist = ['columeastwood', 'OliverDowden']

for i in accountlist:
  # Save output in a variable 
  likestring = "%"+i+"%"
  # Perform a query - this time we grab all fields with *
  QUERY = (
      'SELECT * FROM `tweetsatmps_month1.alltweets`'
      'WHERE tweet LIKE "'+likestring+'"'
      'LIMIT 10')
  query_job = client.query(QUERY)  # API request
  rows = query_job.result()  # Waits for query to finish

  for row in rows:
    #print the whole row
      print(row)

Row(('1491426107333226499', '1414971586071891972', '2022-02-09 14:57:48 UTC', '2022-02-09', '14:57:48', '+0000', '1480298894173089793', 'canrainagain', 'John Rain', None, "@sarahchivers10 @ethaninflux @columeastwood English hate English, go to  football match with 2 local clubs.  Here's a good read, be sure to get to the bit where the Irish Police said not on our doorstep &amp; afterwards Gardai bet the shit out of them !!!   https://t.co/SW7r4nC1lm  https://t.co/N6zjLZZquq", 'en', '[]', "['https://www.theguardian.com/football/2013/may/25/england-ireland-1995-rioting-international']", "['https://pbs.twimg.com/tweet_video_thumb/FLKcIp5XsAEi7dK.jpg']", '1', '0', '0', '[]', '[]', 'https://twitter.com/canrainagain/status/1491426107333226499', 'False', None, '1', 'https://pbs.twimg.com/tweet_video_thumb/FLKcIp5XsAEi7dK.jpg', None, None, None, None, None, None, "[{'screen_name': 'sarahchivers10', 'name': 'sarah chivers', 'id': '1343596632932294657'}, {'screen_name': 'ethaninflux', 'name': 'E

The rows variable is a particular type of bigquery object.

In [ ]:
type(rows)

google.cloud.bigquery.table.RowIterator

## Turning our query into a dataframe

The `bigquery` library integrates with `pandas` - the [documentation includes code on how to generate a dataframe](https://googleapis.dev/python/bigquery/latest/usage/pandas.html) from a SQL query.

In [ ]:
#create a list of accounts
accountlist = ['columeastwood', 'OliverDowden']

for i in accountlist:
  # Save output in a variable 
  likestring = "%"+i+"%"
  # Perform a query.
  QUERY = (
      'SELECT * FROM `tweetsatmps_month1.alltweets`'
      'WHERE tweet LIKE "'+likestring+'"'
      'LIMIT 10')
  #generate a dataframe from that query
  df = client.query(QUERY).to_dataframe()
  print(df.head())

                    ID      conversation_id               created_at  \
0  1491426107333226499  1414971586071891972  2022-02-09 14:57:48 UTC   
1  1491427221617184768  1414971586071891972  2022-02-09 15:02:13 UTC   
2  1491191133753720832  1481969439113416710  2022-02-08 23:24:05 UTC   
3  1488527727942676491  1485622030036869122  2022-02-01 15:00:40 UTC   
4  1489018397878804483  1486393079376527371  2022-02-02 23:30:25 UTC   

         date      time timezone              user_id         username  \
0  2022-02-09  14:57:48    +0000  1480298894173089793     canrainagain   
1  2022-02-09  15:02:13    +0000  1480298894173089793     canrainagain   
2  2022-02-08  23:24:05    +0000  1340662899388366848   tfiddlersgreen   
3  2022-02-01  15:00:40    +0000             80663123  glyniswinestein   
4  2022-02-02  23:30:25    +0000  1372305226657628162  joesmit06875162   

               name place  ...   geo source user_rt_id user_rt retweet_id  \
0         John Rain  None  ...  None   None  

## Generating a CSV for each query

We can use the `.to_csv` function from `pandas` to generate a CSV for a query. 

But how can we generate CSVs with different names?

In [ ]:
#run a query for one MP that exports the results as a CSV
QUERY = (
    'SELECT date, time, timezone, user_id, username, name, tweet, mentions, replies_count, retweets_count, likes_count, hashtags, link, retweet, reply_to FROM `tweetsatmps_month1.alltweets`'
    'WHERE tweet LIKE "%NadineDorries%"')
#generate a dataframe from that query
df = client.query(QUERY).to_dataframe()
#add the account name as a column
df['ataccount'] = "NadineDorries"
print(df.head())
filename = "NadineDorries.csv"
df.to_csv(filename)

         date      time timezone             user_id       username  \
0  2022-02-06  09:29:30    +0000           214188450  sailorsanders   
1  2022-02-01  20:10:28    +0000           464778570      scook2003   
2  2022-02-01  20:10:28    +0000           464778570      scook2003   
3  2022-02-01  20:10:28    +0000           464778570      scook2003   
4  2022-02-05  21:36:50    +0000  951476590910885890  spy_historian   

                name                                              tweet  \
0               Gill  @DavidLammy @NadineDorries @Channel4 Our cultu...   
1  scook2003 💙 #GTTO  @franketteh02 @BorisJohnson @BorisSupporting @...   
2  scook2003 💙 #GTTO  @franketteh02 @BorisJohnson @BorisSupporting @...   
3  scook2003 💙 #GTTO  @franketteh02 @BorisJohnson @BorisSupporting @...   
4        Chris Smith  @ChrisRGollop @Dominic2306 @JohnRentoul @Conse...   

  mentions replies_count retweets_count likes_count hashtags  \
0       []             0              0           0       

In [ ]:
#create a list of accounts
accountlist = ['columeastwood', 'OliverDowden']

for i in accountlist:
  # Save output in a variable 
  likestring = "%"+i+"%"
  # Perform a query.
  QUERY = (
      'SELECT date, time, timezone, user_id, username, name, tweet, mentions, replies_count, retweets_count, likes_count, hashtags, link, retweet, reply_to FROM `tweetsatmps_month1.alltweets`'
      'WHERE tweet LIKE "'+likestring+'"'
      'LIMIT 10')
  #generate a dataframe from that query
  df = client.query(QUERY).to_dataframe()
  #add the account name as a column
  df['ataccount'] = i
  print(df.head())
  filename = i+".csv"
  df.to_csv(filename)

         date      time timezone              user_id         username  \
0  2022-02-09  14:57:48    +0000  1480298894173089793     canrainagain   
1  2022-02-09  15:02:13    +0000  1480298894173089793     canrainagain   
2  2022-02-08  23:24:05    +0000  1340662899388366848   tfiddlersgreen   
3  2022-02-01  15:00:40    +0000             80663123  glyniswinestein   
4  2022-02-02  23:30:25    +0000  1372305226657628162  joesmit06875162   

               name                                              tweet  \
0         John Rain  @sarahchivers10 @ethaninflux @columeastwood En...   
1         John Rain  @sarahchivers10 @ethaninflux @columeastwood Wa...   
2  TheFiddlersgreen  @LoveRascal2 @columeastwood @SDLPlive We had t...   
3   Glynis in Leeds  @MisterTotti @SDLPlive @columeastwood @Nichola...   
4         Joe smith  @JohnnyMercerUK @columeastwood The paratrooper...   

  mentions replies_count retweets_count likes_count hashtags  \
0       []             1              0       

## Download multiple files

We will want to download the results, but not individually. [This Stackoverflow thread]() provides the answer - zip them first.

In [ ]:
!zip -r /content/file.zip /content/

### Create a dedicated folder for our files first

We don't want to zip the 'sample_data' folder that sits alongside our CSVs right now, however, so we need to make a separate folder *just* for our CSVs.

We can do that with the `!mkdir` command in Colab.

In [ ]:
!mkdir querycsvs

## Do this for all MP names

Now we know it works, let's remove the limit and expand our list to all MP account names.

In [ ]:
#import the pandas library first to handle data
import pandas as pd

In [ ]:
#store the CSV url
#oldcsvurl = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQsqzb5L_Y10X7ZJ-r4lMWtZUOA0Dvdt3N64F99X3wsy4_XnbHq-Wj2E2sia10o67HznmDdbcyDpRvM/pub?gid=14619868&single=true&output=csv"
csvurl = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTUxP42PNbJg8koPUcfwOvyBxAs_eI8Q9PqqCxA7hPrPLk16c7tmxd3gmnTsSdvHUZnSSGYUe8zI7bp/pub?gid=0&single=true&output=csv"
#fetch the CSV with MP usernames
mpdf = pd.read_csv(csvurl)
#show the first few rows
mpdf.head(50)

,count,mp_name,mp_surname,on_twitter,account_name,party,constituency,gender
0,1,Diane Abbott,Abbott,yes,@HackneyAbbott,Labour,Hackney North and Stoke Newington,F
1,2,Debbie Abrahams,Abrahams,yes,@Debbie_abrahams,Labour,Oldham East and Saddleworth,F
2,3,Nigel Adams,Adams,yes,@nadams,Conservative,Selby and Ainsty,M
3,4,Bim Afolami,Afolami,yes,@BimAfolami,Conservative,Hitchin and Harpenden,M
4,5,Adam Afriyie,Afriyie,yes,@AdamAfriyie,Conservative,Windsor,M
5,6,Nickie Aiken,Aiken,yes,@twocitiesnickie,Conservative,Cities of London and Westminster,F
6,7,Peter Aldous,Aldous,yes,@peter_aldous,Conservative,Waveney,M
7,8,Rushanara Ali,Ali,yes,@rushanaraali,Labour,Bethnal Green and Bow,F
8,9,Tahir Ali,Ali,yes,@TahirAliMP,Labour,"Birmingham, Hall Green",M
9,10,Lucy Allan,Allan,yes,@lucyallan,Conservative,Telford,F


We can loop through the column of MP account names like this:

In [ ]:
#loop through the first 5 items in that column
for i in mpdf['account_name'][:16]:
  #print it
  print(i)
  print(type(i))
  if isinstance(i, str):
    #remove the @ sign
    print(i.replace("@",""))
  else:
    print("NAN!")

@HackneyAbbott
<class 'str'>
HackneyAbbott
@Debbie_abrahams
<class 'str'>
Debbie_abrahams
@nadams
<class 'str'>
nadams
@BimAfolami
<class 'str'>
BimAfolami
@AdamAfriyie
<class 'str'>
AdamAfriyie
@twocitiesnickie
<class 'str'>
twocitiesnickie
@peter_aldous
<class 'str'>
peter_aldous
@rushanaraali
<class 'str'>
rushanaraali
@TahirAliMP
<class 'str'>
TahirAliMP
@lucyallan
<class 'str'>
lucyallan
@DrRosena
<class 'str'>
DrRosena
@MikeAmesburyMP
<class 'str'>
MikeAmesburyMP
@amessd_southend
<class 'str'>
amessd_southend
@Stuart4WolvesSW
<class 'str'>
Stuart4WolvesSW
@PutneyFleur
<class 'str'>
PutneyFleur
nan
<class 'float'>
NAN!


In [ ]:
#loop through the account names
for i in mpdf['account_name']:
  #remove the @ sign and add % signs for the query
  likestring = "%"+i.replace("@","")+"%"
  # Perform a query to grab tweets naming that account
  QUERY = (
      'SELECT date, time, timezone, user_id, username, name, tweet, mentions, replies_count, retweets_count, likes_count, hashtags, link, retweet, reply_to FROM `tweetsatmps_month1.alltweets`'
      'WHERE tweet LIKE "'+likestring+'"')
  #generate a dataframe from that query
  df = client.query(QUERY).to_dataframe()
  #add the account name as a column
  df['ataccount'] = i
  #generate a string with the name of the account and .csv
  filename = i+".csv"
  #export a CSV with the name of the account, into the dedicated folder
  df.to_csv("querycsvs/"+filename)

This version is for those datasets where the fields aren't named, so instead we need to select all fields where 'string_field_10' contains our matching string. 

In [ ]:
#loop through the account names
for i in mpdf['account_name'][15:]:
  #check if it's a string
  if isinstance(i, str):
    #remove the @ sign and add % signs for the query
    likestring = "%"+i.replace("@","")+"%"
    # Perform a query to grab tweets naming that account
    QUERY = (
        'SELECT * FROM `tweetsatmps_month1.tweetsatmps_mar23_mar30`'
        'WHERE string_field_10 LIKE "'+likestring+'"')
    #generate a dataframe from that query
    df = client.query(QUERY).to_dataframe()
    #add the account name as a column
    df['ataccount'] = i
    #generate a string with the name of the account and .csv
    filename = i+".csv"
    #export a CSV with the name of the account, into the dedicated folder
    df.to_csv("querycsvs/"+filename)
  #if it's not a string just do nothing
  else:
    print("NAN!")

NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!
NAN!


In [ ]:
!zip -r /content/allstuff.zip /content/querycsvs/

  adding: content/querycsvs/ (stored 0%)
  adding: content/querycsvs/@charlotte2153.csv (deflated 76%)
  adding: content/querycsvs/@lisanandy.csv (deflated 84%)
  adding: content/querycsvs/@MattRodda.csv (deflated 82%)
  adding: content/querycsvs/@AngusMacNeilSNP.csv (deflated 79%)
  adding: content/querycsvs/@ChrisM4Chester.csv (deflated 82%)
  adding: content/querycsvs/@Michael4MDNP.csv (deflated 87%)
  adding: content/querycsvs/@NickFletcherMP.csv (deflated 87%)
  adding: content/querycsvs/@JackDromeyMP.csv (deflated 75%)
  adding: content/querycsvs/@SCrabbPembs.csv (deflated 92%)
  adding: content/querycsvs/@SirRogerGale.csv (deflated 89%)
  adding: content/querycsvs/@JamesSunderl.csv (deflated 95%)
  adding: content/querycsvs/@KennyMacAskill.csv (deflated 90%)
  adding: content/querycsvs/@timfarron.csv (deflated 87%)
  adding: content/querycsvs/@patmcfaddenmp.csv (deflated 83%)
  adding: content/querycsvs/@AnneMarieMorris.csv (deflated 90%)
  adding: content/querycsvs/@CPJElmore.c

## Download the resulting zip file


In [ ]:
from google.colab import files
files.download("allstuff.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Code to combine the CSV files for each scrape

We can also use this notebook to run queries we might otherwise run in BigQuery, such as this one to combine the tables from each week's scrape.

In [10]:
# Display query output immediately

%%bigquery --project perspective-api-340114 fulldf
SELECT *
FROM `datacamp-287416.tweetsatmps_month1.tweetsatmps_mar14_mar23_clean`
UNION DISTINCT
SELECT *
FROM `datacamp-287416.tweetsatmps_month1.tweetsatmps_mar23_mar30_clean`
UNION DISTINCT
SELECT *
FROM `datacamp-287416.tweetsatmps_month1.tweetsatmps_mar30_apr06_clean`
UNION DISTINCT
SELECT *
FROM `datacamp-287416.tweetsatmps_month1.tweetsatmps_apr06_13_clean`
UNION DISTINCT
SELECT CAST (id AS string) AS tweet_id,
    CAST (conversation_id AS string) AS conversation_id,
    CAST (created_at AS string) AS created_at,
    CAST (date AS string) AS `date`,
    CAST (time AS string) AS `time`,
    CAST (timezone AS string) AS timezone,
    CAST (user_id AS string) AS user_id,
    username AS username,
    name AS name,
    place AS place,
    tweet AS tweet,
    language AS `language`,
    mentions AS mentions,
    urls AS urls,
    photos AS photos,
    replies_count,
    retweets_count,
    likes_count,
    hashtags,
    link,
    CAST (video AS string) AS video,
    thumbnail,
    reply_to,
FROM `datacamp-287416.tweetsatmps_month1.tweetsatmps_apr13_20` 

In [11]:
len(fulldf)

2846267

In [12]:
fulldf.to_csv("fulldf.csv")
from google.colab import files
files.download("fulldf.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Repeat for the final CSV

In [ ]:
# Display query output immediately

%%bigquery --project perspective-api-340114 fulldf
SELECT *
FROM `datacamp-287416.tweetsatmps_month1.tweetsatmps_mar14_mar23_clean`
UNION DISTINCT
SELECT *
FROM `datacamp-287416.tweetsatmps_month1.tweetsatmps_mar23_mar30_clean`
UNION DISTINCT
SELECT *
FROM `datacamp-287416.tweetsatmps_month1.tweetsatmps_mar30_apr06_clean`
UNION DISTINCT
SELECT *
FROM `datacamp-287416.tweetsatmps_month1.tweetsatmps_apr06_13_clean`
UNION DISTINCT
SELECT CAST (id AS string) AS tweet_id,
    CAST (conversation_id AS string) AS conversation_id,
    CAST (created_at AS string) AS created_at,
    CAST (date AS string) AS `date`,
    CAST (time AS string) AS `time`,
    CAST (timezone AS string) AS timezone,
    CAST (user_id AS string) AS user_id,
    username AS username,
    name AS name,
    place AS place,
    tweet AS tweet,
    language AS `language`,
    mentions AS mentions,
    urls AS urls,
    photos AS photos,
    replies_count,
    retweets_count,
    likes_count,
    hashtags,
    link,
    CAST (video AS string) AS video,
    thumbnail,
    reply_to,
FROM `datacamp-287416.tweetsatmps_month1.tweetsatmps_apr13_20` 
UNION DISTINCT
SELECT CAST (id AS string) AS tweet_id,
    CAST (conversation_id AS string) AS conversation_id,
    CAST (created_at AS string) AS created_at,
    CAST (date AS string) AS `date`,
    CAST (time AS string) AS `time`,
    CAST (timezone AS string) AS timezone,
    CAST (user_id AS string) AS user_id,
    username AS username,
    name AS name,
    place AS place,
    tweet AS tweet,
    language AS `language`,
    mentions AS mentions,
    urls AS urls,
    photos AS photos,
    replies_count,
    retweets_count,
    likes_count,
    hashtags,
    link,
    CAST (video AS string) AS video,
    thumbnail,
    reply_to,
FROM `datacamp-287416.tweetsatmps_month1.tweetsatmps_apr20_27` 

In [ ]:
fulldf.to_csv("fulldf.csv")
from google.colab import files
files.download("fulldf.csv")

More code at https://cloud.google.com/bigquery/docs/dataset-access-controls#python (select the Python tab) and https://googleapis.dev/python/bigquery/latest/index.html